In [ ]:
"%run -i \"D:\\Courses\\2019 Fall (Processing)\\CS-596 Machine Learning\\Assignments\\Assignment 3\\My Code\\main_part1.py\""


In [ ]:
%run -i \"D:\\Courses\\2019 Fall (Processing)\\CS-596 Machine Learning\\Assignments\\Assignment 3\\My Code\\main_part1.py\"


In [54]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from keras.datasets import mnist
from PartI_FNN.util import func_confusion_matrix
from sklearn.model_selection import train_test_split
import time
# import tensorflow as tf
import tensorflow.compat.v1 as tf
from sklearn import preprocessing


# Part I

In [3]:
# load (downloaded if needed) the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# transform each image from 28 by28 to a 784 pixel vector
pixel_count = x_train.shape[1] * x_train.shape[2]
x_train = x_train.reshape(x_train.shape[0], pixel_count).astype('float32')
x_test = x_test.reshape(x_test.shape[0], pixel_count).astype('float32')

# normalize inputs from gray scale of 0-255 to values between 0-1
x_train = x_train / 255
x_test = x_test / 255

# Please write your own codes in responses to the homework assignment 4

In [9]:
# len(x_train)
X_train, X_val, Y_train, Y_val = train_test_split( \
    x_train, y_train, test_size=10000, random_state=34)


### Implement FNN Model Class

In [118]:
# import pickle
# from collections import deque

class FNN_Model:
    """
    neural network model
    """
    def __init__(self, classes_num, IMAGE_PIXELS, params={}):
        self.classes_num = classes_num
        self.image_pixels = IMAGE_PIXELS
        self.LEARNING_RATE = params['LearningRate']
        self.reg = params['reg']

        self.num_hidden = params['HiddenLayerNum']
        self.hidden_size = params['HiddenLayerSize']
        self.log_file_path = params['log_file_path']
        tf.keras.backend.clear_session()
        self.session = self.create_model()

    def define_placeholders(self):
        images_placeholder = tf.placeholder(tf.float32, shape = (None, self.image_pixels))
        labels_placeholder = tf.placeholder(tf.int64, shape = (None, self.classes_num))
        return images_placeholder, labels_placeholder

    def variable_summaries(self,var, name):
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.summary.scalar('sttdev/' + name, stddev)
        tf.summary.scalar('max/' + name, tf.reduce_max(var))
        tf.summary.scalar('min/' + name, tf.reduce_min(var))

    def multilayer_perceptron(self, input_mat):
        n_input = self.image_pixels
        n_hidden_1 = self.hidden_size
        n_hidden_2 = self.hidden_size
        n_classes = self.classes_num

        weights = {
            'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1],stddev = 0.1)),
            'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],stddev = 0.1)),
            'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes],stddev = 0.1))
        }
        biases = {
            'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev = 0.1)),
            'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev = 0.1)),
            'out': tf.Variable(tf.random_normal([n_classes], stddev = 0.1))
        }
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(input_mat, weights['h1']), biases['b1'])
        # mean,variance = tf.nn.moments(layer_1,[0])
        # layer_1 = tf.nn.batch_normalization(layer_1,mean,variance,None,None,0.00001)
        # layer_1 = tf.tanh(layer_1)
        layer_1 = tf.nn.relu(layer_1)
        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        # mean,variance = tf.nn.moments(layer_2,[0])
        # layer_2 = tf.nn.batch_normalization(layer_2,mean,variance,None,None,0.00001)
        # layer_2 = tf.tanh(layer_2)
        layer_2 = tf.nn.relu(layer_2)
        # Output layer with linear activation
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        reg_term = self.reg * (tf.reduce_sum(tf.square(weights['h1']))+
        tf.reduce_sum(tf.square(weights['h2']))+tf.reduce_sum(tf.square(weights['out'])))
        return out_layer, reg_term, weights, biases

    def create_model(self):
        self.images_placeholder, self.labels_placeholder = self.define_placeholders()
        output, reg_term, weights, biases = self.multilayer_perceptron(self.images_placeholder)

        self.variable_summaries(weights['h1'], 'layer1'+ '/weights')
        self.variable_summaries(weights['h2'], 'layer2' + '/weights')
        self.variable_summaries(weights['out'], 'layer3' + '/weights')

        self.NN_output = output

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=self.labels_placeholder))

        tf.summary.scalar('cost', self.cost)

        optimizer = tf.train.GradientDescentOptimizer(learning_rate = self.LEARNING_RATE)
        self.train_op = optimizer.minimize(self.cost)
        init = tf.initialize_all_variables()

        """
        Default runs at cpu,

        For GPU runnning, it takes 33 percent of entire gpu memory
        use this:
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
        session = tf.Session(config=tf.ConfigProto(gpu_options = gpu_options))
        """
        session = tf.Session()
        session.run(init)
        self.saver_op = tf.train.Saver()
        self.merged = tf.summary.merge_all()
        self.train_writer = tf.summary.FileWriter('dqn_log_files/log1/data3',session.graph)
        return session

    def init_t(self):
        self.t = 0

    def save_model(self, path):
        # print(path)
        save_path = self.saver_op.save(self.session, path)
        print ('Model saved in file:%s' %save_path)
        # pass

    def restore_model(self, path):
        self.saver_op.restore(self.session, path)


    def train_step(self, Input_mat, labels):
        _, cost, prediction_probs, self.summary = self.session.run(
        [self.train_op, self.cost, self.NN_output, self.merged],
        feed_dict = {
            self.images_placeholder : Input_mat,
            self.labels_placeholder : labels
        })
        return cost

    def predict(self, states, label):
        cost, prediction_probs = self.session.run(
        [self.cost, self.NN_output],
        feed_dict = {
        self.images_placeholder: states,
        self.labels_placeholder: label
        })
        return prediction_probs

In [ ]:
LEARNING_RATE =  0.001
MAX_STEP = 2000
HIDDEN_LAYER_SIZE = 120
HIDDEN_LAYER_NUM = 2
BATCH_SIZE = 400
TRAIN_FIR = 'tf_logs'
REGULARIZATION = 0.1

# DEFAULT_SAVER_PATH = 'model/'
# DEFAULT_LOG_PATH = "dqn_log/log1/data1"
# DEFAULT_SAVING_FREQUENCY = 100

# # import os
# # import argparse

# DEFAULT_EPI = 10000000
# DEFAULT_STEPS = 500
# DEFAULT_MEM_WIDTH = 100000
# DEFAULT_INITIAL_EPSILON = 1
# DEFAULT_FINAL_EPSILON = 0.1
# DEFAULT_GAMMA = 0.95
# DEFAULT_MINI_BATCH = 16
# DEFAULT_OBSERVATION = 10000

# DEFAULT_LEARNING_RATE = 0.000001
# DEFAULT_REGULARIZATION = 0.001
# DEFAULT_HIDDEN_LAYER = 300
# DEFAULT_HIDDEN_LAYER_NUM = 2

# DEFAULT_SAVER_PATH = 'model/'
# DEFAULT_LOG_PATH = "dqn_log/log1/data1"
# DEFAULT_SAVING_FREQUENCY = 100

# def parse_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-episodes', default = DEFAULT_EPI, type = int)
#     parser.add_argument('-steps', default = DEFAULT_STEPS, type = int )
#     parser.add_argument('-MemSize', default = DEFAULT_MEM_WIDTH, type = int )
#     parser.add_argument('-initial_epsilon', default = DEFAULT_INITIAL_EPSILON, type = int )
#     parser.add_argument('-final_epsilon', default = DEFAULT_FINAL_EPSILON, type = int )
#     parser.add_argument('-gamma', default = DEFAULT_GAMMA, type = int )
#     parser.add_argument('-BatchSize', default = DEFAULT_MINI_BATCH, type = int )
#     parser.add_argument('-reg', default = DEFAULT_REGULARIZATION, type = int )
#     parser.add_argument('-LearningRate', default = DEFAULT_LEARNING_RATE, type = int )
#     parser.add_argument('-HiddenLayerNum', default = DEFAULT_HIDDEN_LAYER_NUM, type = int )
#     parser.add_argument('-HiddenLayerSize', default = DEFAULT_HIDDEN_LAYER, type = int )
#     parser.add_argument('-observation', default = DEFAULT_OBSERVATION, type = int )
#     parser.add_argument('-saver_path', default = DEFAULT_SAVER_PATH, type = str )
#     parser.add_argument('-saving_rate', default = DEFAULT_SAVING_FREQUENCY, type = int)
#     parser.add_argument('-log_path', default = DEFAULT_LOG_PATH, type = str)

#     args = parser.parse_args()
#     training_params = {
#         'saver_path' : args.saver_path,
#         'saving_rate' : args.saving_rate
#     }
#     network_params = {
#         'HiddenLayerSize': args.HiddenLayerSize,
#         'reg':args.reg,
#         'LearningRate': args.LearningRate,
#         'HiddenLayerNum': args.HiddenLayerNum,
#         'mini_batch_size': args.BatchSize,
#         'log_file_path': args.log_path
#         }

#     return training_params, network_params


## Using FNN Model

In [115]:
# OUTPUT_LAYER_SIZE = 2
# INPUT_LAYER_SIZE = 73
# BatchSize = 8

training_epoch = 10
LEARNING_RATE =  0.001
MAX_STEP = 2000
HIDDEN_LAYER_SIZE = 120
HIDDEN_LAYER_NUM = 2
# BATCH_SIZE = 400
BATCH_SIZE = 100
TRAIN_DIR = 'tf_logs'
REGULARIZATION = 0.1

def import_data_training():
    # load (downloaded if needed) the MNIST dataset
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # transform each image from 28 by28 to a 784 pixel vector
    pixel_count = x_train.shape[1] * x_train.shape[2]
    x_train = x_train.reshape(x_train.shape[0], pixel_count).astype('float32')
    # normalize inputs from gray scale of 0-255 to values between 0-1
    x_train = x_train / 255
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=10000, random_state=34)
    
    le = preprocessing.LabelEncoder()
    int_encoded = le.fit_transform(y_train)
    oe = preprocessing.OneHotEncoder(categories='auto')
    oe.fit(int_encoded.reshape(len(int_encoded), 1))
    y_train = oe.transform(y_train.reshape(len(y_train),1)).toarray()
    y_test = oe.transform(y_test.reshape(len(y_test),1)).toarray()

    return x_train, y_train, x_test, y_test
    
def save_nn_model(FNN, SAVING_PATH):
    FNN.save_model(SAVING_PATH+ 'model.ckpt')

def restore_nn_model(FNN, SAVING_PATH):
    ckpt = tf.train.get_checkpoint_state(SAVING_PATH)
    if ckpt and ckpt.model_checkpoint_path:
        FNN.restore_model(ckpt.model_checkpoint_path)
        FNN.init_t()
        print('saved model found and restored')
    else:
        print('No saved model, creating new one...')

def write_to_log(FNN, t):
    FNN.train_writer.add_summary(FNN.summary,t)

In [ ]:
"""load the parameters"""
# SAVING_PATH = training_params['saver_path']
# SAVING_RATE = training_params['saving_rate']
# input_data, output_data = import_data_training()

tf.disable_eager_execution()
x_train, y_train, x_test, y_test = import_data_training()

IMAGE_PIXELS = 784
CLASSES = 10

network_params = {'LearningRate':LEARNING_RATE, 'reg':REGULARIZATION, 'HiddenLayerNum':\
                  HIDDEN_LAYER_NUM,'HiddenLayerSize': HIDDEN_LAYER_SIZE, 'log_file_path':TRAIN_DIR}

FNN = FNN_Model(CLASSES, IMAGE_PIXELS, network_params)

for epoch in range(training_epoch):
    avg_cost = 0
    NumberOfInputs, _ = x_train.shape
    total_batch = int(NumberOfInputs / BATCH_SIZE)

    for i in range(total_batch):
        Input_batch = x_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        Output_batch = y_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        cost = FNN.train_step(Input_batch,Output_batch)
        avg_cost += cost/total_batch
    print("Epoch: {}, cost = {}".format(epoch, avg_cost))
# save_nn_model(FNN, SAVING_PATH)
# test model
Estimated_output = FNN.predict(x_test, y_test)
correct_prediction = np.equal(np.argmax(Estimated_output,1), np.argmax(y_test,1))
# print(correct_prediction)
accuracy = np.sum(correct_prediction.astype(float))/np.size(correct_prediction)
print('Accuracy is {}'.format(accuracy))


Epoch: 0, cost = 2.2348036742210398
Epoch: 1, cost = 2.0383416166305532
Epoch: 2, cost = 1.8308837878704072
Epoch: 3, cost = 1.6014531509876229
Epoch: 4, cost = 1.3732412877082814
Epoch: 5, cost = 1.1730413024425514
Epoch: 6, cost = 1.0128371433019634
Epoch: 7, cost = 0.8903929674625402


In [112]:
le = preprocessing.LabelEncoder()
# le.fit(np.unique(y_train))
le.fit(y_train)
# le.fit(y_train.astype(str))
y_2 = le.transform(y_train)

# y_2.reshape(len(y_2), 1)

oe = preprocessing.OneHotEncoder(categories='auto')
# oe = preprocessing.OneHotEncoder()
oe.fit(y_2.reshape(len(y_2), 1))
oe.transform(y_train.reshape(len(y_train),1)).toarray()



array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

# Temporary Code

# import pickle
# from collections import deque

class FNN:
    """
    neural network model
    """
    def __init__(self, num_actions, STATES_SHAPE, params={}):
        self.num_actions = num_actions
        self.state_shape = STATES_SHAPE
        self.LEARNING_RATE = params['LearningRate']
        self.reg = params['reg']

        self.num_hidden = params['HiddenLayerNum']
        self.hidden_size = params['HiddenLayerSize']
        self.log_file_path = params['log_file_path']
        self.session = self.create_model()

    def define_placeholders(self):
        input_placeholder = tf.placeholder(tf.float32, shape = (None, self.state_shape))
        output_placeholder = tf.placeholder(tf.float32, shape = (None, self.num_actions))
        return input_placeholder, output_placeholder


    def variable_summaries(self,var, name):
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.scalar_summary('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.scalar_summary('sttdev/' + name, stddev)
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))

    def multilayer_perceptron(self, input_mat):
        n_input = self.state_shape
        n_hidden_1 = self.hidden_size
        n_hidden_2 = self.hidden_size
        n_classes = self.num_actions

        weights = {
            'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1],stddev = 0.1)),
            'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],stddev = 0.1)),
            'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes],stddev = 0.1))
        }
        biases = {
            'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev = 0.1)),
            'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev = 0.1)),
            'out': tf.Variable(tf.random_normal([n_classes], stddev = 0.1))
        }
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(input_mat, weights['h1']), biases['b1'])
        # mean,variance = tf.nn.moments(layer_1,[0])
        # layer_1 = tf.nn.batch_normalization(layer_1,mean,variance,None,None,0.00001)
        # layer_1 = tf.tanh(layer_1)
        layer_1 = tf.nn.relu(layer_1)
        # Hidden layer with RELU activation
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        # mean,variance = tf.nn.moments(layer_2,[0])
        # layer_2 = tf.nn.batch_normalization(layer_2,mean,variance,None,None,0.00001)
        # layer_2 = tf.tanh(layer_2)
        layer_2 = tf.nn.relu(layer_2)
        # Output layer with linear activation
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
        reg_term = self.reg * (tf.reduce_sum(tf.square(weights['h1']))+
        tf.reduce_sum(tf.square(weights['h2']))+tf.reduce_sum(tf.square(weights['out'])))
        return out_layer, reg_term, weights, biases

    def create_model(self):
        self.input_placeholder, self.output_placeholder = self.define_placeholders()
        output, reg_term, weights, biases = self.multilayer_perceptron(self.input_placeholder)

        self.variable_summaries(weights['h1'], 'layer1'+ '/weights')
        self.variable_summaries(weights['h2'], 'layer2' + '/weights')
        self.variable_summaries(weights['out'], 'layer3' + '/weights')

        self.NN_output = output

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output,self.output_placeholder))

        tf.scalar_summary('cost', self.cost)

        optimizer = tf.train.GradientDescentOptimizer(learning_rate = self.LEARNING_RATE)
        self.train_op = optimizer.minimize(self.cost)
        init = tf.initialize_all_variables()

        """
        Default runs at cpu,

        For GPU runnning, it takes 33 percent of entire gpu memory
        use this:
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
        session = tf.Session(config=tf.ConfigProto(gpu_options = gpu_options))
        """
        session = tf.Session()
        session.run(init)
        self.saver_op = tf.train.Saver()
        self.merged = tf.merge_all_summaries()
        self.train_writer = tf.train.SummaryWriter('dqn_log_files/log1/data3',session.graph)
        return session

    def init_t(self):
        self.t = 0

    def save_model(self, path):
        # print(path)
        save_path = self.saver_op.save(self.session, path)
        print ('Model saved in file:%s' %save_path)
        # pass

    def restore_model(self, path):
        self.saver_op.restore(self.session, path)


    def train_step(self, Input_mat, actions):
        _, cost, prediction_probs, self.summary = self.session.run(
        [self.train_op, self.cost, self.NN_output, self.merged],
        feed_dict = {
            self.input_placeholder : Input_mat,
            self.output_placeholder : actions
        })
        return cost
        # print(tf.shape(prediction_probs))
        # print(tf.shape(actions))
        #tensorboard --logdir='dqn_log_files/log1/data1'

    def predict(self, states, label):
        cost, prediction_probs = self.session.run(
        [self.cost, self.NN_output],
        feed_dict = {
        self.input_placeholder: states,
        self.output_placeholder: label
        })
        return prediction_probs
